In [1]:
from tools.web_tool import WebTool
import time

In [3]:
import os
import json
import openai
from pydantic import BaseModel, Field
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Create client
client = openai.OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key='85e577a7bd21434e2d3f1ab2bd7a2750c6db5eb7ddf09cce131655911c93f622',
)

In [4]:
from typing import List

class LoS(BaseModel):
    "Get List of strings to complete the given quries"
    los: List[str] = Field(description="List of five strings to complete the provided task")

class SWOTAnalysis(BaseModel):
    "Market share, sales and SWOT (Strength, weakness, opportunity and threat) analysis of given company"
    numbers : List[str] = Field(description="List of key performance indices like market-share, sales, global precense, etc.")
    strength: List[str] = Field(description="List of strenghts the given company have that make them unique, with detailed description")
    weakness: List[str] = Field(description="List of weakness the given company with detailed description")
    opportunity: List[str] = Field(description="List of opportunities the given company have that will contribute in the company future growth with detailed description")
    threats: List[str] = Field(description="List of threats the given company have that can make business go bust, with detailed description")

In [5]:
def get_message(task,**kwargs):
    if task == 'web-query':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your intelligence to complete the task."},
            {"role": "user", "content": f"Web search queries for DuckDuckGo search engine for '{kwargs['query']}'"}
        ]
    elif task == 'competitors-name':
        message = [
            {"role": "system", "content": "You are a helpful and experienced market analyst. Use your intelligence and the provided context(if needed) to complete the task."},
            {"role": "user", "content": f"Name that competitors in the '{kwargs['industry']}' industry.\n\nCONTEXT :\n {kwargs['context']}"}
        ]
    elif task == 'swot-analysis':
        message = [
            {"role": "system", "content": "You are a helpful and experienced market analyst. Use your intelligence the provided context(if needed) to complete the task, dont make things up. Answer in valid JSON"},
            {"role": "user", "content": f"Tell Market share, sales number, growth etc. and SWOT analysis on {kwargs['competitor_name']} as a business.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    return message

In [6]:
def get_response(response_schema,message,repetetion_penalty):
    response = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.1",
        messages=message,
        response_format={"type": "json_object", "schema": response_schema.model_json_schema()},
        frequency_penalty = repetetion_penalty
    )
    content = json.loads(response.choices[0].message.content)
    return content

In [14]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_text)
def main(industry):
    #get competitors name
    message = get_message(task='competitors-name',industry=industry,context='')
    competitors_names = get_response(response_schema=LoS,message=message,repetetion_penalty=0.5)['los']
    final_report = {}
    for competitor in competitors_names[:4]:
        
        queries = [f'Market share, sales number, global presence, of {competitor} in {industry} industry.',
                   f'SWOT Strength, weakness, opportunity and threat of {competitor} in {industry} industry.']
        
        content = remove_stopwords(WebTool().fetch_content(texts=queries))[:4096*4]
        
        clean_content = client.chat.completions.create(
            model='meta-llama/Llama-3-8b-chat-hf',
            messages=[
                {"role": "system", "content": "You are an information retriever and summarizer,ignore everything you know, return only the factual information regarding the QUERY into a maximum of 500-600 words. Output should be concise chunks of paragraphs or tables or both, ignore links, using the RAW TEXT"},
                {"role": "user", "content": f"QUERY :\n\n{','.join(queries)}\n\n RAW TEXT :\n\n{content}"} 
            ],
            frequency_penalty = 0.16)

        
        msg = get_message(task='swot-analysis',competitor_name=competitor,context=clean_content)
        swot_report = get_response(response_schema=SWOTAnalysis,message=msg,repetetion_penalty=0.46)
        final_report[competitor] = swot_report
    return final_report

In [15]:
r = main('Smart home apliances')

/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.mordorintelligence.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.grandviewresearch.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.researchgate.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.

In [16]:
import pandas
pandas.DataFrame(r)

,Nest,Philips Hue,Samsung SmartThings,Apple HomeKit
strength,"[Strong brand recognition, Wide range of produ...","[Strong brand recognition and reputation, Wide...","[Brand recognition, Skilled workforce, Diversi...","[Strong brand recognition, Innovative products..."
weakness,"[High competition, Dependence on Google resour...","[High prices compared to competitors, Dependen...","[Fierce competition, Dependence on smartphone ...","[High prices, Limited compatibility with non-A..."
opportunity,"[Growing demand for smart home solutions, Incr...","[Growing demand for smart home devices, Increa...","[Acquisitions/mergers, Diversification expansi...","[Growing demand for smart home automation, Inc..."
threats,[Competition from established players like Hon...,"[Competition from lower-priced brands, Risk of...","[Increasing competition, Controversy/patent in...","[Competition from other smart home companies, ..."
numbers,[Global Smart Home Market Size: USD 370.95 bil...,"[According to Strategy Analytics, Philips Hue ...",[],"[Market share: 5-7%, Global market size: $80.2..."


In [17]:
import json
from tools.json2html_report import json2html_report
json_response = json.loads(json.dumps(r))
html_content = json2html_report(json_data=json_response,report_title='competition analysis')
with open('x.html','w') as file:
    file.write(html_content)

JSON converted to HTML report
